In [ ]:
import socket
import select

header_length = 10
ip = '127.0.0.1'
port = 9999
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)

server_socket.bind((ip,port))
server_socket.listen()

sockets_list = [server_socket]

clients = {}
def recieve_message(client_socket):
    try:
        message_header = client_socket.recv(header_length)
        
        if not len(message_header):
            return False
        
        message_length = int(message_header.decode('utf-8').strip())
        return {"header":message_header,"data":client_socket.recv(message_length)}
    
    
    except:
        return False
    
while True:
    read_sockets, _,exception_sockets = select.select(sockets_list,[],sockets_list)
    
    for notified_socket in read_sockets:
        if notified_socket == server_socket:
            client_socket,client_address = server_socket.accept()
            
            user = recieve_message(client_socket)
            
            if user is False:
                continue
                
            socket_list.append(client_socket)
            
            clients[client_socket] = user
            
            print(f"Accepted new connection from {client_address[0]}:{client_address[1]} username:{user['data'].decode('utf-8')}")
            
        else:
            message = recieve_message(notified_socket)
                
            if message is False:
                print(f"Closed Connection from {clients[notified_socket]['data'].decode('utf-8')}")
                sockets_list.remove(notified_socket)
                del clients[notified_socket]
                continue
                
            user = clients[notified_socket]
        
            print(f"Recieved message from{user['data'].decode('utf-8')}:{message['data'].decode('utf-8')}")
                
            for client_socket in clients:
                if client_socket != notified_socket:
                    client_socket.send(user['header'] + user['data'] + message['header'] + message['data'])
               
    for notified_socket in exception_sockets:
        sockets_list.remove(notified_socket)
        del clients[notified_socket]
                    
            

In [ ]:
import socket
import select
import errno

header_length = 10
ip = '127.0.0.1'
port = 9999

my_username = input("Username :")
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((ip,port))
client_socket.set_blocking(False)

username = my_username.encode('utf-8')
username_header = f"{len(username):<{header_length}}".encode('utf-8')
client_socket.send(username_header + username)

while True:
    message = input(f"{my_username} > ")
    
    if message:
        message = message.encode('utf-8')
        message_header = f"{len(message) :< {header_length}}".encode('utf-8')
        client_socket.send(message_header + message)
    try:
        while True:
            #recieve things
            username_header = client_socket.recv(header_length)
            if not len((userame_header)):
                print('Connection closed by server')
                sys.exit()
            
            username_length = int(username_length.decode('utf-8').strip())
            username = client_socket.recv(username_length).decode('utf-8')
            
            message_header = client_socket.recv(header_length)
            message_length = int(message_header.decode('utf-8').strip())
            message = client_socket.recv(message_length).decode('utf-8')
            
            print(f"{username} > {message}")
            
    except IOError as e:
        if e.errno != errno.EAGAIN and e.errno != errno.EWOULDBLOCK:
            print('reading error',str(e))
            sys.exit()
        continue
    
    
    except Exception as e:
        print('Error',str(e))
        sys.exit()

In [ ]:
""